In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, CuDNNGRU, Bidirectional, Masking
from keras.layers import Dense, InputLayer, Dropout, BatchNormalization
from keras.layers import Reshape, Flatten, Conv1D, AveragePooling1D, Activation
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, Callback
import keras.backend as K
from sklearn.model_selection import train_test_split

from tqdm import tqdm, trange, tnrange
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [35]:
from data_gen import create_weekly_kplus, create_credit_seq, create_daily_kplus, create_demographic_data
from util import modified_SMAPE
from losses import SMAPE_loss
from callbacks import EvaluateSMAPE

load data gen


In [4]:
demographics = pd.read_csv('datasets/exam-1/demographics.csv')
cc = pd.read_csv('datasets/exam-1/cc.csv')
cc.sort_values(by=['cc_no', 'pos_dt'], inplace=True)
kplus = pd.read_csv('datasets/exam-1/kplus.csv')
kplus.sort_values(by=['id', 'sunday'], inplace=True)
train_set = pd.read_csv('datasets/exam-1/train.csv')
test_set = pd.read_csv('datasets/exam-1/test.csv')

In [5]:
raw_demographics = pd.read_csv('datasets/exam-1/raw_demograhgics.csv')

In [6]:
train_scaler = StandardScaler()
scaled_train_set = train_set.copy()
scaled_train_set.set_index('id', inplace=True)
scaled_train_set['income'] = train_scaler.fit_transform(np.expand_dims(scaled_train_set['income'], axis=2))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


In [44]:
xs = create_demographic_data(raw_demographics[raw_demographics['id'] <= 50000])

In [39]:
ys = scaled_train_set.loc[:50001]['income'].to_numpy()

In [40]:
xs.shape, ys.shape

((500, 21), (50000,))

In [46]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [51]:
model = Sequential()
model.add(InputLayer((21,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

In [52]:
model.compile(optimizer='adam', loss=SMAPE_loss(train_scaler))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                704       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 1,249
Trainable params: 1,249
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(x_train, y_train,
         validation_data=(x_val, y_val),
          epochs=32
         )

Train on 40000 samples, validate on 10000 samples
Epoch 1/32
40000/40000 [==============================] - 2s 38us/step - loss: 0.1050 - val_loss: 0.1039
Epoch 2/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1032 - val_loss: 0.1033
Epoch 3/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1030 - val_loss: 0.1031
Epoch 4/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1030 - val_loss: 0.1037
Epoch 5/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1028 - val_loss: 0.1036
Epoch 6/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1027 - val_loss: 0.1031
Epoch 7/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1027 - val_loss: 0.1035
Epoch 8/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1027 - val_loss: 0.1033
Epoch 9/32
40000/40000 [==============================] - 1s 31us/step - loss: 0.1026 - val_loss: 0.1036
Epoch

KeyboardInterrupt: 